In [2]:
import os
from ultralytics import YOLO
from ultralytics.nn.modules.conv import *
from ultralytics.nn.modules.block import *
import torch

In [3]:
model = YOLO(r'D:\malaria prediction\parasite_v1\final_results\kaggle_ver14\working\runs\detect\train\weights\last.pt')
model_p2 = YOLO(r'D:\malaria prediction\parasite_v1\final_results\kaggle_ver15\working\runs\detect\train\weights\last.pt')
model_p2_temp = YOLO('yolo11s-p2.yaml')
model.model.model


Sequential(
  (0): Conv(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
  (1): Conv(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
  (2): C3k2(
    (cv1): Conv(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (cv2): Conv(
      (conv): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (m): ModuleList(
      (0): Bottleneck(
        (cv1): Conv(
     

In [ ]:
# model_p2.model.model[1].load_state_dict(model.model.model[1].state_dict())

<All keys matched successfully>

In [5]:
def compare_models(model_a, model_b):
    differences = {}
    for (name_a, param_a), (name_b, param_b) in zip(model_a.state_dict().items(), model_b.state_dict().items()):
        if not torch.equal(param_a, param_b):
            differences[name_a] = (param_a - param_b).abs().max().item()
    if differences:
        print("Models have differences in weights at the following layers:")
        for layer, max_diff in differences.items():
            print(f"Layer {layer}: Max difference {max_diff}")
    else:
        print("No differences in weights; models are identical.")
    return 
compare_models(model.model.model, model_p2.model.model)

No differences in weights; models are identical.


In [32]:
def load_block(block1, block2, strict=False):
    try:
        missing_keys, unexpected_keys = block1.load_state_dict(block2.state_dict(), strict=strict)
        print("State dictionary transferred successfully!")
        
        if missing_keys:
            print("Warning: The following keys are missing in the destination model:", missing_keys)
        if unexpected_keys:
            print("Warning: The following unexpected keys are present in the source model:", unexpected_keys)

    except Exception as e:
        print(f"Error transferring state_dict: {e}")

def load_model(model1, model2, dict_matches):
    for key, value in dict_matches.items():
        print(f"Loading block {key} and {value}...")
        load_block(model1[key], model2[value], strict=False)
    


In [33]:
backbone_matches = {
    i:i for i in range(11)
}
backbone_matches

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10}

In [34]:
neck_matches = {
    **{i: i for i in range(11, 17)},  # Matches from 11 to 16
    23: 17,
    24: 18,
    25: 19,
    26: 20,
    27: 21,
    28: 22
}
neck_matches

{11: 11,
 12: 12,
 13: 13,
 14: 14,
 15: 15,
 16: 16,
 23: 17,
 24: 18,
 25: 19,
 26: 20,
 27: 21,
 28: 22}

In [35]:
head_matches = {29:23}

In [36]:
load_model(model_p2.model.model, model.model.model, 
           {**backbone_matches, **neck_matches, **head_matches})



Loading block 0 and 0...
State dictionary transferred successfully!
Loading block 1 and 1...
State dictionary transferred successfully!
Loading block 2 and 2...
State dictionary transferred successfully!
Loading block 3 and 3...
State dictionary transferred successfully!
Loading block 4 and 4...
State dictionary transferred successfully!
Loading block 5 and 5...
State dictionary transferred successfully!
Loading block 6 and 6...
State dictionary transferred successfully!
Loading block 7 and 7...
State dictionary transferred successfully!
Loading block 8 and 8...
State dictionary transferred successfully!
Loading block 9 and 9...
State dictionary transferred successfully!
Loading block 10 and 10...
State dictionary transferred successfully!
Loading block 11 and 11...
State dictionary transferred successfully!
Loading block 12 and 12...
State dictionary transferred successfully!
Loading block 13 and 13...
State dictionary transferred successfully!
Loading block 14 and 14...
State diction

In [41]:
compare_models(model_p2.model.model, model_p2_temp.model.model)

Models have differences in weights at the following layers:
Layer 0.conv.weight: Max difference 0.3692026138305664
Layer 1.conv.weight: Max difference 0.11612878739833832
Layer 2.cv1.conv.weight: Max difference 0.24822865426540375
Layer 2.cv2.conv.weight: Max difference 0.20357948541641235
Layer 2.m.0.cv1.conv.weight: Max difference 0.11646291613578796
Layer 2.m.0.cv2.conv.weight: Max difference 0.16537591814994812
Layer 3.conv.weight: Max difference 0.05872444063425064
Layer 4.cv1.conv.weight: Max difference 0.1760936677455902
Layer 4.cv2.conv.weight: Max difference 0.14389541745185852
Layer 4.m.0.cv1.conv.weight: Max difference 0.0825783759355545
Layer 4.m.0.cv2.conv.weight: Max difference 0.11668649315834045
Layer 5.conv.weight: Max difference 0.041618913412094116
Layer 6.cv1.conv.weight: Max difference 0.12485718727111816
Layer 6.cv2.conv.weight: Max difference 0.10194937139749527
Layer 6.m.0.cv1.conv.weight: Max difference 0.17498548328876495
Layer 6.m.0.cv2.conv.weight: Max diffe

In [42]:
model_p2.save('yolo11s-p2_malaria_loaded_pretrain.pt')

TypeError: 'NoneType' object is not a mapping